In [1]:
import numpy as np
import tensorflow as tf
import uncertainty_metrics.tensorflow as umtf
import uncertainty_metrics.numpy as umnp
import tensorflow_probability as tfp

In [2]:
pred_prob = np.array([0.7,0.67,0.23,0.19,0.9,0.97,0.54,0.77,0.15,0.3,0.85,0.69,0.34,0.45,0.53])
pred_prob2= np.array([[0.3,0.7],[0.3,0.67],[0.77,0.23],[0.81,0.19],[0.1,0.9],[0.03,0.97],[0.46,0.54],[0.23,0.77],[0.85,0.15],[0.7,0.3],[0.15,0.85],[0.31,0.69],[0.64,0.34],[0.55,0.45],[0.47,0.53]])

pred_class = np.array([1,1,0,0,1,1,1,1,0,0,1,1,0,0,1])
true_class = np.array([1,0,1,0,1,0,0,0,1,0,1,1,0,1,1])

In [4]:
pred_prob_cl = np.array([0.66666667, 0.5, 0.4, 0.4, 0.66666667,0.66666667, 0.5, 0.66666667, 0.4, 0.4 ,
       0.66666667, 0.66666667, 0.4, 0.5, 0.5])
pred_prob_cl

array([0.66666667, 0.5       , 0.4       , 0.4       , 0.66666667,
       0.66666667, 0.5       , 0.66666667, 0.4       , 0.4       ,
       0.66666667, 0.66666667, 0.4       , 0.5       , 0.5       ])

In [5]:
# this takes the multiclass format of taking the max probability
# ece1 = umtf.ExpectedCalibrationError(true_class, pred_prob, num_bins=5)
ece2 = umnp.ece(true_class, pred_prob, num_bins=5)
ece2_cl = umnp.ece(true_class, pred_prob_cl, num_bins=5)

print(ece2,ece2_cl)


0.19733333333333344 1.3333333548359862e-09


In [174]:
#https://towardsdatascience.com/expected-calibration-error-ece-a-step-by-step-visual-explanation-with-python-code-c3e9aa12937d
def expected_calibration_error(samples, true_labels, M=3,binary=True):
    # uniform binning approach with M number of bins
    bin_boundaries = np.linspace(0, 1, M + 1)
    bin_lowers = bin_boundaries[:-1]
    bin_uppers = bin_boundaries[1:]

   # # keep confidences / predicted "probabilities" as they are
   #  confidences = samples

    if binary:
        # keep confidences / predicted "probabilities" as they are
        confidences = samples
        # get binary predictions from confidences
        predicted_label = (samples>0.5).astype(float)
    else:                                          
        # get max probability per sample i                 
        confidences = np.max(samples, axis=1)               
        # get predictions from confidences (positional in this case)
        predicted_label = np.argmax(samples, axis=1).astype(float)

    # get a boolean list of correct/false predictions
    accuracies = predicted_label==true_labels

    ece = np.zeros(1)
    for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
        # determine if sample is in bin m (between bin lower & upper)
        in_bin = np.logical_and(confidences >= bin_lower.item(), confidences < bin_upper.item())
        # can calculate the empirical probability of a sample falling into bin m: (|Bm|/n)
        prop_in_bin = in_bin.astype(float).mean()

        if prop_in_bin.item() > 0:
            # get the accuracy of bin m: acc(Bm)
            accuracy_in_bin = accuracies[in_bin].astype(float).mean()
            # get the average confidence of bin m: conf(Bm)
            avg_confidence_in_bin = confidences[in_bin].mean()
            # calculate |acc(Bm) - conf(Bm)| * (|Bm|/n) for bin m and add to the total ECE
            ece += np.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin
    return ece

In [128]:
expected_calibration_error(pred_prob,true_class,5)

array([0.25733333])

In [129]:

expected_calibration_error(pred_prob2,true_class,5,False)

array([0.196])

In [134]:
# ece=tfp.stats.expected_calibration_error(
#     num_bins=5, logits=pred_prob, labels_true=true_class, labels_predicted=pred_class, name=None
# )

### Multiclass

In [185]:
pred_prob_mult = np.array([[0.7,0.2,0.1],[0.1,0.67,0.23],[0.82,0.13,0.05],[0.82,0.03,0.15],
                           [0.05,0.05,0.9],[0.97,0.01,0.02],[0.3,0.54,0.16],[0.2,0.77,0.03],
                           [0.15,0.1,0.75],[0.6,0.1,0.3],[0.05,0.85,0.1],[0.25,0.06,0.69],
                           [0.4,0.34,0.26],[0.42,0.45,0.13],[0.3,0.17,0.53]],np.float32)
true_class = np.array([1,0,1,2,1,0,1,2,2,0,1,2,0,1,0])
pred_class = np.array([0,1,0,0,2,0,1,1,2,0,1,2,0,1,2])

In [186]:
ece2 = umnp.ece(true_class, pred_prob_mult, num_bins=5)
print(ece2)


0.3080000023047129


In [175]:
expected_calibration_error(pred_prob_mult,true_class,5,False)

array([0.308])

In [159]:
    binary=False
    bin_boundaries = np.linspace(0, 1, 5 + 1)
    bin_lowers = bin_boundaries[:-1]
    bin_uppers = bin_boundaries[1:]

   # # keep confidences / predicted "probabilities" as they are
   #  confidences = samples

    if binary:
        # keep confidences / predicted "probabilities" as they are
        confidences = pred_prob_mult
        # get binary predictions from confidences
        predicted_label = (pred_prob_mult>0.5).astype(float)
    else:                                          
        # get max probability per sample i                 
        confidences = np.max(pred_prob_mult, axis=1)               
        # get predictions from confidences (positional in this case)
        predicted_label = np.argmax(pred_prob_mult, axis=1).astype(float)

    # get a boolean list of correct/false predictions
    accuracies = predicted_label==true_class


In [160]:
predicted_label

array([0., 1., 0., 0., 2., 0., 1., 1., 2., 0., 1., 2., 0., 1., 2.])

In [161]:
pred_class

array([0, 1, 0, 0, 2, 0, 1, 1, 2, 0, 1, 2, 0, 1, 2])

In [162]:
confidences

array([0.7 , 0.67, 0.82, 0.82, 0.9 , 0.97, 0.54, 0.77, 0.75, 0.6 , 0.85,
       0.69, 0.4 , 0.45, 0.53], dtype=float32)

In [176]:
    ece = np.zeros(1)
    for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
        # determine if sample is in bin m (between bin lower & upper)
        in_bin = np.logical_and(confidences >= bin_lower.item(), confidences < bin_upper.item())
        # can calculate the empirical probability of a sample falling into bin m: (|Bm|/n)
        prop_in_bin = in_bin.astype(float).mean()
        print(in_bin,prop_in_bin)

        if prop_in_bin.item() > 0:
            # get the accuracy of bin m: acc(Bm)
            accuracy_in_bin = accuracies[in_bin].astype(float).mean()
            # get the average confidence of bin m: conf(Bm)
            avg_confidence_in_bin = confidences[in_bin].mean()
            # calculate |acc(Bm) - conf(Bm)| * (|Bm|/n) for bin m and add to the total ECE
            print("conf",np.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin)
            ece += np.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin

[False False False False False False False False False False False False
 False False False] 0.0
[False False False False False False False False False False False False
 False False False] 0.0
[False False False False False False  True False False False False False
  True  True  True] 0.26666666666666666
conf 0.07199999491373697
[ True  True False False False False False  True  True  True False  True
 False False False] 0.4
conf 0.07866666316986085
[False False  True  True  True  True False False False False  True False
 False False False] 0.3333333333333333
conf 0.1573333462079366


In [164]:
ece

array([0.36133332])

In [165]:
bin_lowers, bin_uppers

(array([0. , 0.2, 0.4, 0.6, 0.8]), array([0.2, 0.4, 0.6, 0.8, 1. ]))

In [170]:
  b = np.linspace(start=0, stop=1.0, num=6)
  bins = np.digitize(confidences, bins=b, right=True)

In [171]:
b

array([0. , 0.2, 0.4, 0.6, 0.8, 1. ])

In [172]:
bins

array([4, 4, 5, 5, 5, 5, 3, 4, 4, 4, 5, 4, 3, 3, 3], dtype=int64)

In [173]:
confidences


array([0.7 , 0.67, 0.82, 0.82, 0.9 , 0.97, 0.54, 0.77, 0.75, 0.6 , 0.85,
       0.69, 0.4 , 0.45, 0.53], dtype=float32)

## SCE

In [207]:
pred_prob_mult_1 = np.array([[0.7,0.2,0.1],[0.1,0.67,0.23],[0.82,0.13,0.05],[0.82,0.03,0.15],\
                         [0.05,0.05,0.9],[0.97,0.01,0.02],[0.3,0.54,0.16],[0.2,0.77,0.03],\
                         [0.15,0.1,0.75],[0.6,0.1,0.3],[0.05,0.85,0.1],[0.25,0.06, 0.69],\
                         [0.4,0.34,0.26],[0.42,0.45,0.13],[0.3,0.17,0.53]],np.float64)

pred_prob_mult_2 = np.array([[0.7,0.2,0.1],[0.1,0.67,0.23],[0.82,0.13,0.05],[0.82,0.03,0.15],
                           [0.05,0.05,0.9],[0.97,0.01,0.02],[0.3,0.54,0.16],[0.2,0.77,0.03],
                           [0.15,0.1,0.75],[0.6,0.1,0.3],[0.05,0.85,0.1],[0.25,0.06,0.69],
                           [0.4,0.34,0.26],[0.42,0.45,0.13],[0.3,0.17,0.53]],np.float32)

pred_prob_mult_3 = np.array([[0.7,0.2,0.1],[0.1,0.67,0.23],[0.82,0.13,0.05],[0.82,0.03,0.15],
                           [0.05,0.05,0.9],[0.97,0.01,0.02],[0.3,0.54,0.16],[0.2,0.77,0.03],
                           [0.15,0.1,0.75],[0.6,0.1,0.3],[0.05,0.85,0.1],[0.25,0.06,0.69],
                           [0.4,0.34,0.26],[0.42,0.45,0.13],[0.3,0.17,0.53]])


In [204]:

true_class = np.array([1,0,1,2,1,0,1,2,2,0,1,2,0,1,0])
pred_class = np.array([0,1,0,0,2,0,1,1,2,0,1,2,0,1,2])

In [208]:
ece1 = umnp.ece(true_class, pred_prob_mult_1, num_bins=5)
ece2 = umnp.ece(true_class, pred_prob_mult_2, num_bins=5)
ece3 = umnp.ece(true_class, pred_prob_mult_3, num_bins=5)

print(ece1,ece2,ece3)


0.3613333333333333 0.3080000023047129 0.3613333333333333


In [201]:
expected_calibration_error(pred_prob_mult,true_class,5,False)

array([0.36133333])

In [219]:
umnp.sce(true_class, pred_prob_mult_2, num_bins=5)

0.24977777703768678

In [221]:
umnp.sce(true_class, pred_prob_mult_1, num_bins=5)

0.26755555555555555

In [222]:
umnp.ace(true_class, pred_prob_mult_1, num_bins=5)

0.2773333333333334

In [220]:
umtf.sce(true_class, pred_prob_mult_1, num_bins=5)

AttributeError: module 'uncertainty_metrics.tensorflow' has no attribute 'sce'

In [223]:

def get_adaptive_bins(predictions, num_bins):
  """Returns upper edges for binning an equal number of datapoints per bin."""
  if np.size(predictions) == 0:
    return np.linspace(0, 1, num_bins+1)[:-1]

  edge_indices = np.linspace(0, len(predictions), num_bins, endpoint=False)

  # Round into integers for indexing. If num_bins does not evenly divide
  # len(predictions), this means that bin sizes will alternate between SIZE and
  # SIZE+1.
  edge_indices = np.round(edge_indices).astype(int)

  # If there are many more bins than data points, some indices will be
  # out-of-bounds by one. Set them to be within bounds:
  edge_indices = np.minimum(edge_indices, len(predictions) - 1)

  # Obtain the edge values:
  edges = np.sort(predictions)[edge_indices]

  # Following the convention of numpy.digitize, we do not include the leftmost
  # edge (i.e. return the upper bin edges):
  return edges[1:]

In [283]:
  edge_indices = np.linspace(0, len(predictions), 5, endpoint=False)

  # Round into integers for indexing. If num_bins does not evenly divide
  # len(predictions), this means that bin sizes will alternate between SIZE and
  # SIZE+1.
  edge_indices = np.round(edge_indices).astype(int)

  # If there are many more bins than data points, some indices will be
  # out-of-bounds by one. Set them to be within bounds:
  edge_indices = np.minimum(edge_indices, len(predictions) - 1)

  # Obtain the edge values:
  edges = np.sort(predictions)[edge_indices]

  # Following the convention of numpy.digitize, we do not include the leftmost
  # edge (i.e. return the upper bin edges):
  # return edges[1:]
edges[1:]

array([0.3 , 0.53, 0.69, 0.85])

In [280]:
  edge_indices = np.linspace(0, len(predictions), num_bins, endpoint=False)

  # Round into integers for indexing. If num_bins does not evenly divide
  # len(predictions), this means that bin sizes will alternate between SIZE and
  # SIZE+1.
  edge_indices = np.round(edge_indices).astype(int)

  # If there are many more bins than data points, some indices will be
  # out-of-bounds by one. Set them to be within bounds:
  edge_indices = np.minimum(edge_indices, len(predictions) - 1)
edge_indices

array([ 0,  3,  6,  9, 12])

In [224]:
pred_prob

array([0.7 , 0.67, 0.23, 0.19, 0.9 , 0.97, 0.54, 0.77, 0.15, 0.3 , 0.85,
       0.69, 0.34, 0.45, 0.53])

In [227]:
bin_upper_bounds = get_adaptive_bins(pred_prob2, 5)
bin_upper_bounds


array([[0.19, 0.81],
       [0.46, 0.54],
       [0.3 , 0.7 ],
       [0.34, 0.64]])

In [228]:
bin_upper_bounds = get_adaptive_bins(pred_prob, 5)
bin_upper_bounds

array([0.3 , 0.53, 0.69, 0.85])

In [229]:
predictions=pred_prob
num_bins=5

In [230]:
  if np.size(predictions) == 0:
    print(np.linspace(0, 1, num_bins+1)[:-1])

In [244]:
edge_indices = np.linspace(0, len(predictions), 5, endpoint=False)
edge_indices

array([ 0.,  3.,  6.,  9., 12.])

In [245]:
  # Round into integers for indexing. If num_bins does not evenly divide
  # len(predictions), this means that bin sizes will alternate between SIZE and
  # SIZE+1.
edge_indices = np.round(edge_indices).astype(int)
edge_indices

array([ 0,  3,  6,  9, 12])

In [240]:
15/6

2.5

In [246]:
  # If there are many more bins than data points, some indices will be
  # out-of-bounds by one. Set them to be within bounds:
  edge_indices = np.minimum(edge_indices, len(predictions) - 1)
edge_indices

array([ 0,  3,  6,  9, 12])

In [247]:
  # Obtain the edge values:
  edges = np.sort(predictions)[edge_indices]
edges

array([0.15, 0.3 , 0.53, 0.69, 0.85])

In [248]:
np.sort(predictions)

array([0.15, 0.19, 0.23, 0.3 , 0.34, 0.45, 0.53, 0.54, 0.67, 0.69, 0.7 ,
       0.77, 0.85, 0.9 , 0.97])

In [253]:
edges[1:]

array([0.3 , 0.53, 0.69, 0.85])

In [ ]:
b1=0 to 0.3
b2=0.3 to 0.53
b3=0.53 to 0.69
b4=0.69 to 0.85
b5=0.85 to 1

In [272]:
edges=np.array([0.3 , 0.53, 0.69, 0.85,1])

In [273]:
bin_indices = np.digitize(predictions, edges)
bin_indices

array([3, 2, 0, 0, 4, 4, 2, 3, 0, 1, 4, 3, 1, 1, 2], dtype=int64)

In [257]:
      bin_upper_bounds = np.histogram_bin_edges(
          [], bins=num_bins, range=(0.0, 1.0))[1:]
bin_upper_bounds

array([0.2, 0.4, 0.6, 0.8, 1. ])

In [259]:
predictions

array([0.7 , 0.67, 0.23, 0.19, 0.9 , 0.97, 0.54, 0.77, 0.15, 0.3 , 0.85,
       0.69, 0.34, 0.45, 0.53])

In [258]:
bin_indices = np.digitize(predictions,bin_upper_bounds)
bin_indices

array([3, 3, 1, 0, 4, 4, 2, 3, 0, 1, 4, 3, 1, 2, 2], dtype=int64)

In [291]:
edges = np.append(edges,np.array([1.]))
edges

array([0.15, 0.3 , 0.53, 0.69, 0.85, 1.  ])

In [2]:
def get_bin_upper_bound(predictions,num_bins,binning_scheme="even"):
    if binning_scheme=="even":
        bin_upper_bounds = np.histogram_bin_edges([], bins=num_bins, range=(0.0, 1.0))[1:]
    elif binning_scheme=="adaptive":
        edge_indices = np.linspace(0, len(predictions), num_bins, endpoint=False)
        # Round into integers for indexing. If num_bins does not evenly divide
        # len(predictions), this means that bin sizes will alternate between SIZE and
        # SIZE+1.
        edge_indices = np.round(edge_indices).astype(int)
    
        # If there are many more bins than data points, some indices will be
        # out-of-bounds by one. Set them to be within bounds:
        edge_indices = np.minimum(edge_indices, len(predictions) - 1)
    
        # Obtain the edge values:
        edges = np.sort(predictions)[edge_indices]

        # add the 1. to the upper bound
        edges = np.append(edges,np.array([1.]))

        # Following the convention of numpy.digitize, we do not include the leftmost
        # edge (i.e. return the upper bin edges):
        bin_upper_bounds=edges[1:]
        
            
    return bin_upper_bounds

In [ ]:
bin_upper_bound_ece = 

In [266]:
bb=np.linspace(0, 1, num_bins+1)[1:]
bb

array([0.2, 0.4, 0.6, 0.8, 1. ])

In [267]:
bin_indices = np.digitize(predictions,bb)
bin_indices

array([3, 3, 1, 0, 4, 4, 2, 3, 0, 1, 4, 3, 1, 2, 2], dtype=int64)

In [ ]:
array([3, 3, 1, 0, 4, 4, 2, 3, 0, 1, 4, 3, 1, 2, 2], dtype=int64)


In [294]:
b = np.linspace(start=0, stop=1.0, num=5)
b = np.quantile(predictions, b)
b = np.unique(b)
num_bins = len(b)

In [295]:
num_bins

5

In [296]:
b = np.linspace(start=0, stop=1.0, num=5)
b

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [297]:
b = np.quantile(predictions, b)
b

array([0.15 , 0.32 , 0.54 , 0.735, 0.97 ])

In [299]:
np.sort(predictions)

array([0.15, 0.19, 0.23, 0.3 , 0.34, 0.45, 0.53, 0.54, 0.67, 0.69, 0.7 ,
       0.77, 0.85, 0.9 , 0.97])

In [300]:
b = np.unique(b)
b

array([0.15 , 0.32 , 0.54 , 0.735, 0.97 ])

In [305]:
pred_prob_ace = np.array([0.7,0.67,0.77,0.81,0.9,0.97,0.54,0.77,0.85,0.7,0.85,0.69,0.66,0.55,0.53])
pred_prob_ace

array([0.7 , 0.67, 0.77, 0.81, 0.9 , 0.97, 0.54, 0.77, 0.85, 0.7 , 0.85,
       0.69, 0.66, 0.55, 0.53])

In [306]:
true_class = np.array([1,0,1,0,1,0,0,0,1,0,1,1,0,1,1])

In [328]:
umnp.ace(true_class, pred_prob_mult_2, num_bins=5)

0.3266666656980913

In [329]:
umnp.sce(true_class, pred_prob_mult, num_bins=5)

0.3177777777777778

In [326]:
umnp.ece(true_class, pred_prob_mult_2, num_bins=5)

0.30800000230471297

In [320]:
pred_prob

array([0.7 , 0.67, 0.23, 0.19, 0.9 , 0.97, 0.54, 0.77, 0.15, 0.3 , 0.85,
       0.69, 0.34, 0.45, 0.53])

In [321]:
true_class

array([1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1])

In [330]:
pred_prob_ace = np.array([0.7,0.67,0.77,0.81,0.9,0.97,0.54,0.77,0.85,0.7,0.85,0.69,0.66,0.55,0.53])
np.sort(pred_prob_ece)

array([0.53, 0.54, 0.55, 0.66, 0.67, 0.69, 0.7 , 0.7 , 0.77, 0.77, 0.81,
       0.85, 0.85, 0.9 , 0.97])

In [332]:
bin_upper_bounds=get_bin_upper_bound(pred_prob_ace,num_bins,binning_scheme="adaptive")
bin_upper_bounds

array([0.66, 0.7 , 0.77, 0.85, 1.  ])

In [334]:
b = np.linspace(start=0, stop=1.0, num=5)
b = np.quantile(pred_prob_ace, b)
b

array([0.53 , 0.665, 0.7  , 0.83 , 0.97 ])

In [5]:
pred_prob_cl = np.array([0.5 , 0.4 , 0.5 , 0.5 , 0.75, 0.75, 0.4 , 0.5 , 0.75, 0.5 , 0.75,
       0.5, 0.4, 0.4, 0.4 ])
bin_upper_bounds=get_bin_upper_bound(pred_prob_cl,5,binning_scheme="adaptive")
bin_upper_bounds

array([0.4 , 0.5 , 0.5 , 0.75, 1.  ])

In [6]:
np.sort(pred_prob_cl)

array([0.4 , 0.4 , 0.4 , 0.4 , 0.4 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 ,
       0.75, 0.75, 0.75, 0.75])